# Youtube spam detection

Download [link](http://www.dt.fee.unicamp.br/~tiago//youtubespamcollection/)

In [1]:
from fastai2.text.all import *
from lazylabel.text.all import *

In [2]:
source = Path('/home/lgvaz/.data/youtube_spam/')

In [3]:
fns = get_files(source, extensions='.csv')
dfs = [pd.read_csv(fn) for fn in fns]

In [4]:
df = pd.concat(dfs)
print(df.size); df.head()

9780


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z12pgdhovmrktzm3i23es5d5junftft3f,lekanaVEVO1,2014-07-22T15:27:50,i love this so much. AND also I Generate Free Leads on Auto Pilot &amp; You Can Too! http://www.MyLeaderGate.com/moretraffic﻿,1
1,z13yx345uxepetggz04ci5rjcxeohzlrtf4,Pyunghee,2014-07-27T01:57:16,http://www.billboard.com/articles/columns/pop-shop/6174122/fan-army-face-off-round-3 Vote for SONES please....we're against vips....please help us.. &gt;.&lt;﻿,1
2,z12lsjvi3wa5x1vwh04cibeaqnzrevxajw00k,Erica Ross,2014-07-27T02:51:43,"Hey guys! Please join me in my fight to help abused/mistreated animals! All fund will go to helping pay for vet bills/and or helping them find homes! I will place an extra emphasis on helping disabled animals, ones otherwise would just be put to sleep by other animal organizations. Donate please. http://www.gofundme.com/Angels-n-Wingz﻿",1
3,z13jcjuovxbwfr0ge04cev2ipsjdfdurwck,Aviel Haimov,2014-08-01T12:27:48,http://psnboss.com/?ref=2tGgp3pV6L this is the song﻿,1
4,z13qybua2yfydzxzj04cgfpqdt2syfx53ms0k,John Bello,2014-08-01T21:04:03,Hey everyone. Watch this trailer!!!!!!!! http://believemefilm.com?hlr=h2hQBUVB﻿,1


In [5]:
df['original'] = df['CONTENT']

In [6]:
cder,tkzer,nmzer = mk_transform(ColReader('original')),Tokenizer.from_df('CONTENT'),Numericalize()

In [7]:
ABSTAIN,SPAM,HAM = 'abstain','spam','ham'
vocab = [ABSTAIN,SPAM,HAM]

In [8]:
labeller = Labeller()
@labeller(tkzer)
def check(x): return SPAM if 'check' in x else ABSTAIN
@labeller(cder)
def check_out(x): return SPAM if 'check out' in x.lower() else ABSTAIN

In [9]:
tls = TfmdLists(df, [cder,tkzer,nmzer])
tasks = tasks_labels(labeller, tls, vocab)

In [11]:
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [12]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.83% (400),1
check,24.32% (467),1


In [13]:
def get_sentences(tls, idxs):
    sentences = {}
    for i in idxs:
        tks = tls.decode(tls[i])
        sentences[i] = (' '.join(decode_spec_tokens(tks.split(' '))))
    return pd.DataFrame(sentences.values(), sentences.keys())

In [15]:
idxs = labeller.find(dls.train, vocab, ['check', 'check_out'], [SPAM, ABSTAIN])

In [16]:
get_sentences(tls, idxs).sample(10)

,0
1329,xxbos ┏ ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ┏┏┏ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ╋╋╋ ┏┏┏ ┓┓┓ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ ┃┃┃ ┃┃┃ ╋╋╋ 444 ┃┃┃ ┗┗┗ ┓┓┓ ┗┗┗ ┛┛┛ ┏┏┏ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ ┃┃┃ ╋╋╋ 444 ┃┃┃ ╋╋╋ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┏┏┏ ┛┛┛ ┗┗┗ ┛┛┛ ┃┃┃ ╋╋╋ ╋╋╋ ┃┃┃ ┃┃┃ ┗┗┗ 333 ━━━ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ 333 ━━━ ┛┛┛ ╋╋╋ ╋╋╋ ┗┗┗ ┛┛┛ CHECK MY VIDEOS AND SUBSCRIBE AND LIKE XXUNK
159,"xxbos a friend of mine has xxunk a big dick xxunk . He had a small dick ( 4 xxunk ) and he did some xxunk about this xxunk . Xxunk the xxunk , he found out the secret xxunk of xxunk xxunk . He xxunk what he had learned and now he has a 7 xxunk dick . He was absolutely xxunk by his xxunk . Of xxunk , it took a few months . Xxunk , he has xxunk a book about this xxunk in order to help guys . He xxunk me to xxunk his book :) So , guys if you are xxunk and for more xxunk check this out https : / / xxunk / b / xxunk . He is xxunk it for xxunk xxunk 1 . Thank you ; ) ﻿"
740,xxbos i think about 100 millions of the views come from people who only wanted to check the views﻿
91,xxbos Can you check my videos please ? Do n't hate me :( Give me one xxunk
907,xxbos I 'm here to check the views .. holy xxunk
46,"xxbos She named the tiger Xxunk Xxunk No , seriously , she did , check the video ﻿"
680,"xxbos Hey Music Fans i really appreciate any of you who will take the time to read this , and check my music out ! I&'m just a 15 year old boy DREAMING of being a successful MUSICIAN in the music world . i do lots of covers , and piano covers . But i don&'t have money to advertise . a simple thumbs up to my comment , a comment on my videos or a SUBSCRIPTION would be a step forward ! It will only be a few seconds of your life that you won&'t regret !!! Thank u to all the people who just give me a chance ! :)"
34,"xxbos Hello ! I 'm Xxunk , I 'm a singer from Xxunk ! i was part of a boy - girl band named xxunk , and I 'm now singing on my own xxunk i wanted to play my own pop and pop - rock songs . It would mean a lot if you could have a look at my channel to check my music and watch my new video ! ! and if u like , subscribe to it ! XOXO THANKS ! ! XXUNK : if you like a lot my channel , you can share it with your friends ! ! Haha ! ! LOL XXUNK"
861,xxbos check men out i put xxunk of xxunk into my music but xxunk not many watch it﻿
801,xxbos Xxunk it you just came here to check the number of xxunk ﻿


In [17]:
@labeller(cder)
def regex_check_out(x):
    return SPAM if re.search(r'check.*out', x, flags=re.I) else ABSTAIN

In [18]:
@labeller(cder)
def regex_check(x):
    return SPAM if re.search(r'(check\s(my|this))|check\s(\w+)?\s?out', x, flags=re.I) else ABSTAIN

In [20]:
tasks = tasks_labels(labeller, tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [21]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.42% (392),1
regex_check_out,22.45% (431),1
regex_check,23.07% (443),1
check,23.91% (459),1


In [22]:
idxs = labeller.find(dls.train, vocab, ['regex_check', 'regex_check_out'], [SPAM, ABSTAIN])
print(len(idxs))
get_sentences(tls, idxs).sample(10)

17


,0
1329,xxbos ┏ ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ┏┏┏ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ╋╋╋ ┏┏┏ ┓┓┓ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ ┃┃┃ ┃┃┃ ╋╋╋ 444 ┃┃┃ ┗┗┗ ┓┓┓ ┗┗┗ ┛┛┛ ┏┏┏ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ ┃┃┃ ╋╋╋ 444 ┃┃┃ ╋╋╋ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┏┏┏ ┛┛┛ ┗┗┗ ┛┛┛ ┃┃┃ ╋╋╋ ╋╋╋ ┃┃┃ ┃┃┃ ┗┗┗ 333 ━━━ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ 333 ━━━ ┛┛┛ ╋╋╋ ╋╋╋ ┗┗┗ ┛┛┛ CHECK MY VIDEOS AND SUBSCRIBE AND LIKE XXUNK
209,"xxbos Rap from Xxunk , check my xxunk"
14,xxbos https : / / soundcloud.com / xxunk please check my stuff ; and make some xxunk
756,xxbos Check my channel please ! And listen to the best music ever : p﻿
880,xxbos CHECK MY CHANNEL FOR MY NEW SONG ' xxunk ' ! ! YOU 'll LOVE XXUNK
804,xxbos Check my channel﻿
940,"xxbos I m a RAPPER / SONGWRITER , check my video please .. also subscribe for more thanks :) tell me what you xxunk"
764,"xxbos if you like xxunk talent , xxunk lyrics , xxunk real hip hop Everyone check my newest sound Xxunk x - Got the Xxunk ( prod by . Xxunk the Model Xxunk ) COMMENT TELL ME WHAT YOU THINK DO nt BE XXUNK !!!! ---- 1111 //// 7777 Xxunk"
224,xxbos want to win xxunk the xxunk - xxunk ? check my channel : ) ﻿
729,xxbos and u xxunk check my channel and tell me what i should do xxunk


In [24]:
idxs = labeller.find(dls.train, vocab, ['regex_check'], [SPAM])
get_sentences(tls, idxs).sample(10)

,0
1891,"xxbos Check out our app to xxunk all your party / drunk xxunk ! \n < a href=""https : / / play.google.com / store / apps / details?id = xxunk = xxunk : / / play.google.com / store / apps / details?id = xxunk = xxunk < / a>﻿"
1012,xxbos Hi guys my name is Xxunk and i do XXUNK football videos i have xxunk subscribers and i think you guys would like my content so come check it out and if you do subscribe!﻿
669,"xxbos Hey guys and girls check out Xxunk Xxunk for hilarious you tube videos , xxunk , and xxunk xxunk !"
139,xxbos Hey guys plz check out my youtube channel to c funny 2 girls 1 cup xxunk thanks and plz subscribe ! Xxunk
1264,"xxbos You guys should check out this EXTRAORDINARY website called ZONEPA.COM . You can make money online and start working from home today as i am ! i am making over $ 3 , 000 +++ perperper monthmonthmonth atatat ZONEPA.COM ! Visit Zonepa.com and check it out ! The xxunk xxunk the xxunk xxunk . The xxunk top xxunk the opinion . The xxunk xxunk xxunk the xxunk ."
1510,xxbos DO YOU KNOW HOW XXUNK XXUNK GOT FAMOUS WHY DON&'T YOU LOOK IT UP KID BEFORE YOUR SO HARD ON YOURSELF ! ! IF YOU HIT ME UP WITH a MESSAGE IN MY XXUNK AND SUBSCRIBE i WILL CHECK OUT YOUR CHANNEL … xxunk XXUNK TO ME .
1378,xxbos could you guys please check out my channel for xxunk beats ?
64,xxbos GO TO MY CHANNEL and check out my xxunk songs﻿
1445,xxbos eminem new song check out my videos
803,xxbos Check my first video xxunk


If instead of having a recursive modification of __call__ we adopt a list of listeners, we can generate all stuff for the preprocessor first and then pass wherever it's necessary. This would use a for loop

In [ ]:
@labeller(cder, TextBlob)
def tb_polarity(x, tb):
    return HAM if tb.polarity > 0.9 else ABSTAIN
@labeller(cder, TextBlock)
def tb_sentiment(x, tb):
    return HAM if tb.subjectivity >= 0.5 else ABSTAIN

In [21]:
from textblob import TextBlob

In [22]:
text = df.iloc[1291]['original']
print(text)
scores = TextBlob(text)

You guys should check out this EXTRAORDINARY website called ZONEPA.COM . You can make money online and start working from home today as I am! I am making over $3,000+ per month at ZONEPA.COM ! Visit Zonepa.com and check it out! How does the mammoth waste achieve the shock? How does the limit reduce the delicate minute? How does the meaty scale adapt the oil?


In [23]:
scores.polarity, scores.subjectivity

(0.050347222222222175, 0.6333333333333333)